# Chapter 2: Individual security returns

Based on [Analyzing Financial Data and Implementing Financial Models Using R](https://www.amazon.com/Analyzing-Financial-Implementing-Springer-Economics/dp/3030641546) by Clifford S. Ang (Springer), but we're using Python instead of R.

This notebook is based on the first edition. There's a newer second edition of the book but I don't have it.

In [1]:
import pandas as pd
import yfinance as yf

# Cache API calls
import requests_cache

In [2]:
# FIXME Temporarily pinning pandas to 1.3.5 since 1.4.0 breaks Ticker.history() in yfinance 0.1.69.
# See https://github.com/ranaroussi/yfinance/issues/937. After yfinance updates this, we can remove
# the pandas version pin.
pd.__version__

'1.3.5'

In [3]:
yf.__version__

'0.1.69'

In [4]:
session = requests_cache.CachedSession("yfinance.cache")
session.headers["User-Agent"] = "prices-notebook/1.0"

# Realty Income REIT
# Pays monthly dividends.
o = yf.Ticker("O", session=session)

## 2.1 Price returns

The price return is the percentage change in the closing price of a security. For example, if a security goes from \\$100 to \\$120 over some time period, then the price return is 20%.

Price returns include capital gains, and exclude intermediate cash flows such as stock dividends or bond coupons.

In [5]:
o_df = o.history(start="2021-12-01", end="2021-12-31")
o_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-12-01,68.214882,68.852683,65.673645,65.693573,6009300.0,0.000,0
2021-12-02,66.321416,67.826224,66.191858,67.288078,5086200.0,0.000,0
2021-12-03,67.347873,67.646838,66.600449,67.019005,4045500.0,0.000,0
2021-12-06,67.537221,69.341009,67.477429,67.846161,5604700.0,0.000,0
2021-12-07,68.165058,68.643405,67.806294,68.155090,6229000.0,0.000,0
2021-12-08,68.115228,68.852684,68.005605,68.613510,4298400.0,0.000,0
2021-12-09,68.284643,68.284643,67.108696,67.128632,3697900.0,0.000,0
2021-12-10,67.577082,67.577082,66.660244,66.710075,3873900.0,0.000,0
2021-12-13,66.759901,68.234821,66.620382,67.925888,4984500.0,0.000,0


In [6]:
# Remove the last row, which was for a dividend.
# Keep only the close column since that's what we use to calculate price returns.
o_df = o_df["2021-12-01":"2021-12-30"][["Close"]]
o_df

,Close
Date,
2021-12-01,65.693573
2021-12-02,67.288078
2021-12-03,67.019005
2021-12-06,67.846161
2021-12-07,68.155090
2021-12-08,68.613510
2021-12-09,67.128632
2021-12-10,66.710075
2021-12-13,67.925888


In [7]:
# Calculate price return.
# Pandas has a ready-made pct_change() function for this.
# NOTE: This procedure does _not_ account for stock splits.
o_df["PriceReturn"] = o_df["Close"].pct_change()
o_df

,Close,PriceReturn
Date,,
2021-12-01,65.693573,NaN
2021-12-02,67.288078,0.024272
2021-12-03,67.019005,-0.003999
2021-12-06,67.846161,0.012342
2021-12-07,68.155090,0.004553
2021-12-08,68.613510,0.006726
2021-12-09,67.128632,-0.021641
2021-12-10,66.710075,-0.006235
2021-12-13,67.925888,0.018225


In [8]:
# Cleanup
o_df = o_df["2021-12-02":"2021-12-30"][["PriceReturn"]]
o_df

,PriceReturn
Date,
2021-12-02,0.024272
2021-12-03,-0.003999
2021-12-06,0.012342
2021-12-07,0.004553
2021-12-08,0.006726
2021-12-09,-0.021641
2021-12-10,-0.006235
2021-12-13,0.018225
2021-12-14,-0.012764


## 2.2 Total returns

Also known as _holding period returns_. This includes both the capital gain and intermediate cash flows.

## 2.3 Logarithmic total returns 

## 2.4 Cumulating multi-day returns 

## 2.5 Weekly returns

## 2.6 Monthly returns

## 2.7 Comparing performance of multiple securities: total returns